In [27]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Pandas가 문자열을 자르지 않도록 설정
pd.set_option('display.max_colwidth', None)

# CSV 파일 경로
input_file_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_euc_kr.csv'
output_file_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_1.csv'

# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv(input_file_path, encoding='euc-kr')  # euc-kr로 인코딩된 CSV 파일을 읽음

# Chrome 웹 드라이버 설정
chrome_path = '/Users/jeonghyeonjeong/Documents/4-1/CAPSTONE/chromedriver-mac-x64/chromedriver'  # chromedriver의 실제 경로로 변경
chrome_service = ChromeService(chrome_path)
browser = webdriver.Chrome(service=chrome_service)

# 빈 리스트 생성
post_contents = []

# 행 범위 지정
start_row = 0  # 시작 행
end_row = 10  # 종료 행 (종료 행을 포함하려면 +1 해야 함)

# 각 행에 대해 반복
for index, row in df.iloc[start_row:end_row + 1].iterrows():
    url = row['Post Link']
    
    browser.get(url)

    try:
        # 10초 동안 기다려서 특정 요소가 로드될 때까지 대기
        element_present = EC.presence_of_element_located((By.CLASS_NAME, 'write_div'))
        WebDriverWait(browser, 10).until(element_present)

        # 페이지의 HTML 내용을 가져오기
        page_source = browser.page_source

        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(page_source, 'html.parser')

        # 클래스가 "write_div"인 div 태그 찾기
        write_div_tag = soup.find('div', class_='write_div')
        
        # div 태그 내에 있는 내용 추출
        post_content = write_div_tag.text.strip() if write_div_tag else None

        # 내용이 있으면 리스트에 추가
        if post_content:
            post_contents.append(post_content)
        else:
            print(f"{url}에서 글 내용을 찾을 수 없습니다.")
    except Exception as e:
        print(f"{url}에서 콘텐츠를 검색하는 데 실패했습니다. 오류: {e}")

# 브라우저 닫기
browser.quit()

# 새로운 열 'Post Content'를 생성하여 내용 추가
df['Post Content'] = post_contents

# DataFrame을 CSV 파일로 저장 (euc-kr로 인코딩)
df.to_csv(output_file_path, index=False, encoding='euc-kr')


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.71)
Stacktrace:
0   chromedriver                        0x00000001055ed918 chromedriver + 4794648
1   chromedriver                        0x00000001055e4ea3 chromedriver + 4759203
2   chromedriver                        0x00000001051be88d chromedriver + 407693
3   chromedriver                        0x00000001051a366e chromedriver + 296558
4   chromedriver                        0x00000001051cbd32 chromedriver + 462130
5   chromedriver                        0x0000000105250b97 chromedriver + 1006487
6   chromedriver                        0x0000000105234a73 chromedriver + 891507
7   chromedriver                        0x00000001051ff133 chromedriver + 672051
8   chromedriver                        0x000000010520030e chromedriver + 676622
9   chromedriver                        0x00000001055ae675 chromed

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Read the CSV file
file_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_euc_kr.csv'
df = pd.read_csv(file_path, encoding='euc-kr')

# Specify the range of rows you want to crawl (e.g., rows 0 to 499)
start_row = 2
end_row = 501

# Create a new list to store the scraped data
result_data = []

# Function to scrape content from the given URL
def scrape_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content_div = soup.find('div', class_='write_div')
    if content_div:
        return content_div.get_text(strip=True)
    else:
        return None

# Loop through each row in the specified range of rows in the original DataFrame
for index, row in df.iloc[start_row:end_row + 1].iterrows():
    post_link = row['Post Link']
    content = scrape_content(post_link)
    
    # Append the data to the result list
    result_data.append({'Post Link': post_link, 'Content': content})

# Convert the list to a DataFrame
result_df = pd.DataFrame(result_data)

# Save the result DataFrame to a new CSV file
result_file_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_1.csv'
result_df.to_csv(result_file_path, index=False)

print(f"Scraping completed. Data saved to {result_file_path}")


Scraping completed. Data saved to /Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_1.csv


In [4]:
from selenium import webdriver
import logging

# 로그 출력을 활성화
logging.basicConfig(level=logging.INFO)

# Chrome WebDriver 초기화
driver = webdriver.Chrome()

# 로그 출력을 확인하기 위한 웹 페이지 열기
driver.get('https://gall.dcinside.com/mgallery/board/view/?id=centristpolitics&no=1481722')


In [2]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Selenium WebDriver를 초기화합니다. ChromeDriver를 사용하는 예시입니다.
driver = webdriver.Chrome()

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 5부터 10까지인 데이터만 필터링
filtered_df = df[df['id'].between(1, 100)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    driver.get(link)
    
    # 페이지 로딩을 위해 충분한 시간을 주세요.
    driver.implicitly_wait(10)
    
    # 페이지의 HTML을 가져오기
    page_content = driver.page_source
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # div.writing_view_box 부분을 찾아서 가져오기
    writing_view_box = soup.find('div', class_='writing_view_box')
    
    # 가져온 내용을 출력
    if writing_view_box:
        content = writing_view_box.text.strip()
        print(f"Content of {link}:")
        print(content)
        
        # 가져온 내용을 리스트에 추가
        contents_list.append(content)
    else:
        print(f"div.writing_view_box not found in {link}")
        contents_list.append(None)  # 없는 경우에는 None 추가

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장 (UTF-8로 인코딩)
filtered_df.to_csv('/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_filtered_with_contents.csv', index=False, encoding='utf-8')

# Selenium을 종료합니다.
driver.quit()


The chromedriver version (119.0.6045.105) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4094042:
국민의힘은 11일 자체 의뢰 여론조사에서 구리·김포·하남 시민 10명 중 6명 이상이 서울시 편입에 찬성하는 것으로 나타났다고 밝혔다. 국민의힘 '뉴시티 프로젝트 특별위원회'는 여론조사업체 <에이스리서치>에 의뢰해 지난 4∼5일 해당지역 성인(구리 803명·김포 812명·하남 804명)을 대상으로 여론조사 결과를 실시한 결과 이같이 나타났다고 밝혔다. 서울편입 여론은 구리가 67%로 가장 높았고, 김포 61%, 하남 60%였다. 반대는 하남 37%, 김포 36%, 구리 29% 순으로 나타났다. 조경태 특위 위원장은 "메가시티를 하게 되면 생활환경이 개선되고, 삶의 질에 긍정적 영향이 있을 것이라는 기대도 모두 60%를 넘었다"면서 "그간 우리가 추진한 '뉴시티 프로젝트'는 지금까지 전례를 찾을 수 없는, 주민을 위한 정책임이 드러났다"고 주장했다. 이번 여론조사는 무선 100% 자동응답(ARS) 방식으로 진행됐고, 표본오차는 95% 신뢰수준에서 ±3.5%포인트였다. 응답률은 각각 구리 5.4%, 김포 6.1%, 하남 5.7%였다. http://www.viewsnnews.com/article?q=216171 [에이스리서치] 구리-김포-하남 60%이상 "서울편입 찬성"구리 67%, 김포 61%, 하남 60%www.viewsnnews.com - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=joongdo1&no=501739:
https://www.viewsnnews.com/article?q=216171 [에이스리서치] 구리-김포-하남 60%이상 "서울편입 찬성"구리 67%, 김포 61%, 하남 60%www.viewsnnews.com - dc official App
Content of https://gall.dc

Content of https://gall.dcinside.com/mgallery/board/view/?id=whdnjswls&no=203:
우리공화당 조원진 대표 “대한민국이 도약할 수 있는 길은 ‘수도권 규제 철폐-지방 투자 법인세 면제’라는 정책을 동시에 써 경제성장과 지방분권을 달성해야”   국민의힘이 ‘메가 서울’ 구상을 불쑥 내놓았다. 서울특별시가 김포시를 편입하겠다는 선언이다. 2024년 총선용으로 보이지만, 실현 여부에 따라 수도권 집중, 지방분권, 수도권 집값 등에 큰 영향을 줄 것으로 보여 그 파급력이 예사롭지 않을 전망이다. 김기현 국민의힘 대표는 김포뿐만 아니라 다른 서울 인접 도시들도 ‘주민이 원할 경우’ 편입을 추진하겠다는 뜻을 밝혔다.  1 이미지 순서 ON이 와중에 서울이, 아니 수도권이 얼마나 ‘좋은 일자리’를 많이 갖고 있는지를 알 수 있는 통계가 발표되었다. 국내 근로소득자의 상위 1% 10명 가운데 8명은 수도권에 거주하는 것으로 나타났다. 이들의 평균 급여는 3억1700만원으로 조사되었다. 수도권 인구가 전체의 절반이 안되는데 고소득자 비중은 80% 이상으로, 수도권에 대한민국의 최고의 일자리와 인재가 몰려있다는 사실을 반영하고 있다.   서울의 김포 편입은 언뜻 보면 수도권 집중을 강화시킬 것으로 보인다. 저출산과 축소경제(슈링코노믹스: shrink+economics) 도래로 지방은 소멸의 위기를 맞이하고 있는 상황에서 이러한 지방분권에 역행하는 국민의힘의 ‘역발상’은 지방을 궤멸시킬 것처럼 공포를 자아낸다. 지금까지 수도권의 ‘확대와 집중’은 중앙정부의 지방분권론에 막혀 구상은 여러 번 있었지만 실현된 적은 없었다.   우리공화당은 국민의힘의 ‘메가 서울’ 구상이 선거용 전략으로 나온 감이 있지만, 이를 계기로 지방분권정책을 획기적으로 바꾸는 기회로 삼을 것을 제안한다. 조원진 당대표는 “국민의힘이 갑작스럽게 서울의 김포 편입을 선언한 것은 ‘총선을 위한 예정된 수순’이라고 말하면서, 그렇지만 이를 계기로 ‘수도권도 살

Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4090521:
https://view.asiae.co.kr/article/2023120708192382632중장기 종합 대책·연구에 학계도 깜짝1 이미지 순서 ON"저출산 대책 관련 수많은 보고서가 있었지만, 정책 효과를 구체적인 숫자로 제시한 것은 처음이다. 예산 당국을 움직이려면 정량적인 근거가 있어야 하는데 한국은행 보고서가 반가운 이유다."(대통령 직속 저출산·고령사회위원회 홍석철 상임위원)한국은행이 지난 3일 내놓은 경제전망보고서 '초저출산과 초고령사회 극단적 인구구조 원인·영향·대책' 중장기 심층연구에 대한 각계각층의 호평이 쏟아지고 있다. 첫 번째 반응은 그동안 저출산 영역에 크게 관심을 기울이지 않았던 한은이 1년이 넘는 기간 동안 깊이 있는 연구를 진행했다는 사실에 깜짝 놀랐다는 것이다. 두 번째 반응은 저출산 관련 주제별·부분적으로 진행된 기존 연구와 달리 종합적으로 각 대책에 대한 출산율 제고 효과를 구체적인 숫자로 보여줘 정책 당국에 시사하는 바가 크다는 점이다.2 이미지 순서 ON◆정책효과 숫자로 제시한 첫 보고서= 지난해 기준 한국의 합계출산율은 0.78명이다. 보고서에 따르면 한국의 청년층 고용률(15~39세, 58%)이 경제협력개발기구(OECD) 34개국 평균 수준(66.6%)으로 증가하면 출산율이 0.12명 올라 0.90명이 된다. 또 10.3주에 불과한 한국의 육아휴직 실이용기간을 OECD 평균 수준인 61.4주로 늘리면 출산율이 0.1명 높아져 출산율이 1.0명대로 반등할 수 있다. 산술적으로 여러 정책 중 한국의 도시인구집중과 혼외출산비중을 OECD 평균 수준에 맞추면 출산율은 각각 0.41명, 0.16명이나 급증해 효과가 크지만, 이 정책은 현실적으로 단기간 변화되기 어렵다는 점에서 한은은 청년 고용률을 높이고 남성의 육아휴직 실이용기간을 늘리는 것이 현 시점에서 보다 실

Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4088077:
경기 김포시는 이달 중순께 행정안전부에 주민투표를 실시하기 위한 건의서를 전달할 예정이라고 7일 밝혔다. 주민투표는 지난 11월16일 국민의힘 국회의원들이 발의한 '경기도와 서울시간 관할구역 변경에 관한 특별법' 제정을 위한 법적 절차다. 지방자치법에 따르면 지방자치단체의 단위를 변경할 때는 주민투표나 지방의회 의견 청취를 거쳐야 한다. 김포시는 지난 1995년 김포 검단면이 인천시로 편입된 기존 사례를 참고해 김포 주민만을 대상으로 한 주민투표를 실시해야 주장하고 있다. 1995년 주민투표는 검단면 주민만을 대상으로 진행됐다. 김포시는 현재 김포시민 1000명을 대상으로 한국갤럽에 의뢰해 서울 편입 관련 대면 여론조사를 진행 중이다. 여론조사는 이달 말까지 진행된다. 김포시가 지난달 시민 1010명을 대상으로 실시한 유선 자동응답(ARS) 비대면 방식의 여론조사에서는 68%가 서울 편입에 ‘찬성’한다고 답한 바 있다. 당시 여론조사의 응답률은 6.1%, 표본오차는 95% 신뢰수준에 ±3.1%p였다. 김포시는 또 오는 15일 김포아트홀에서 최순종 경기대 행정복지상담대학원장을 좌장으로 서울 편입과 관련한 토론회도 열 예정이다. 김포시 관계자는 "이달 안으로 행정안전부에 주민투표 실시를 위한 건의서를 전달할 계획이다"며 "직접 행안부를 찾는 대면방식으로 의견을 전달할 지 검토 중이다"고 말했다. https://n.news.naver.com/article/421/0007221270
Content of https://gall.dcinside.com/mgallery/board/view/?id=ohsehoon&no=10234:
https://www.mbn.co.kr/news/society/4984876 김포, 서울 편입 행정절차 밟기 시작…주민투표 건의 계획 경기도 김포시가 서울특별시에 편입되기 위한 행정

...

Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2253362:
식은 떡밥이라고 생각했는데, 정작 전화를 받고 나니 아직 덜 식은 느낌이라 올린다.오늘 여론조사 전화 받았는데, 어떻게든 서울 편입 찬성률 올리려고 발악하는 게 느껴진다.경기도 분도는 간단하게 설명하고, 서울 편입은 긍정적인 내용을 중심으로 길게 설명하는 것부터 의도가 느껴졌음.만약 선택지에 "현행대로 유지"가 있었으면 "서울 편입"보다 더 많이 선택할 거라고 본다.계속해서 김포시민들이 서울 편입을 원한다는 조사가 나올텐데 대부분 이렇게 오염된 문항이라고 보면 됨.아래는 조사 전문임. 에이닷 덕분에 수월하게 정리할 수 있었음.현재 경기도에서는 지역균형 발전을 위해 경기도를 경기북도와 경기남도로 구분해 경기북부특별자치도를 설치하는 방안을 추진중입니다.한편 여당은 생활권과 행정구역이 달라 벌어지는 교통, 의료, 교육, 문화 등의 주민 불편을 개선하고 행정 효율을 위해 서울과 인접한 경기 지역 일부를 편입하는 내용의 메가시티를 추진중인데요, 이와 관련해 몇 가지 질문 드리겠습니다.1. 선생님께서는 만약 현재 살고 계시는 지역이 서울로 편입되는 것에 대해 어떻게 생각하십니까?1) 매우 긍정적으로 생각한다. 2) 다소 긍정적으로 생각한다. 3) 다소 부정적으로 생각한다. 4) 매우 부정적으로 생각한다. 5) 잘 모르겠다.2. 선생님께서는 만약 현재 살고 계시는 지역이 서울로 편입될 경우에 교통, 의료, 주요 행정, 각종 인프라 등 생활환경 개선 면에서 어떠할 것으로 보십니까?1) 매우 도움이 될 것 같다. 2) 다소 도음이 될 것 같다. 3) 별로 도움이 안 될 것 같다. 4) 전혀 도움이 안 될 것 같다. 5) 잘 모르겠다.3. 그러시다면 선생님께서는 서울로의 편입이 선생님 본인의 생활이나 삶의 질 측면에서는 어떤 영향이 있을 것이로 보십니까?1) 매우 긍정적 영향이 있을 것이다. 2) 다소 긍정적 영향이 있을 것이다. 

Content of https://gall.dcinside.com/board/view/?id=fantasy_new2&no=5085084:
해달라고- dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=alliescon&no=2135176:
1 이미지 순서 ON2 이미지 순서 ONhttps://n.news.naver.com/article/449/0000263001?sid=102 [단독]서울시, ‘기후동행카드’에 김포시 참여 검토서울시가 내년 1월 도입하는 대중교통 무제한 이용권인 '기후동행카드'에 김포시 참여 여부를 검토하고 있습니다. 지난달 6일 김병수 김포시장이 오세훈 서울시장을 찾아 김포 편입을 공식 요청한 이후 '기후동행카드' 참여n.news.naver.com 총선 기대해도 좋다 가즈아~~~ 3 이미지 순서 ON
Content of https://gall.dcinside.com/mgallery/board/view/?id=joongdo1&no=493541:
https://n.news.naver.com/article/449/0000263001?sid=102 [단독]서울시, ‘기후동행카드’에 김포시 참여 검토서울시가 내년 1월 도입하는 대중교통 무제한 이용권인 '기후동행카드'에 김포시 참여 여부를 검토하고 있습니다. 지난달 6일 김병수 김포시장이 오세훈 서울시장을 찾아 김포 편입을 공식 요청한 이후 '기후동행카드' 참여n.news.naver.com - dc official App
Content of https://gall.dcinside.com/board/view/?id=drama_new3&no=12728317:
https://naver.me/GABTs6p5 ‘메가 서울’ 구상에… 김포·구리·하남 시민 과반이 찬성■ 문화일보·케이스탯 서울편입 여론조사 ‘50만명 이내 도시 편입’ 관련 광명은 찬성 47% - 반대 50% 여권이 추진하는 ‘메가 서울’ 구상에 대해 경기 김포·구

Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4084351:
■ 문화일보·케이스탯 서울편입 여론조사 68%가 긍정적 응답 ‘압도적’ 수도권 중소도시 ‘욕구’ 확인 문화일보가 케이스탯리서치에 의뢰해 실시한 여론조사에서 김포뿐 아니라 구리·하남·광명 주민들도 본인 거주 지역의 서울 편입에 찬성하는 여론이 절반을 넘는 것으로 나타났다. 경기도 중소도시의 서울시 편입 자체가 적절한지에 대해선 찬반이 엇갈리고 있지만, 적어도 이들 중소도시 거주민의 서울 편입 ‘욕구’는 이번 조사를 통해 입증됐다는 평가가 나온다. 4일 여론조사 결과에 따르면 김포시 서울 편입에 대한 김포 지역 응답자의 찬성률이 57%로 절반을 넘어선 가운데, 구리시 서울 편입에 대한 구리 지역 응답자의 찬성률은 이보다도 11%포인트가 높은 68%에 달하는 것으로 조사됐다. 반대하는 응답자의 비율은 29%에 그쳤고, 연령별·권역별·이념 성향별 대부분의 계층에서 찬성론이 우세했다. 광명에서는 광명시 서울 편입에 찬성한다는 응답이 55%로, 반대한다는 응답 43%를 웃돌았다. 60대(69%)와 70대(82%)에서 찬성론이, 20대(48%)와 40대(57%)에서 반대론이 높게 나왔다. 하남에서도 하남시 서울 편입에 찬성한다는 응답은 57%로 반대(40%)를 크게 앞섰다. 네 지역 모두에서 본인 거주지가 서울에 합쳐지는 데 대해선 긍정적 반응이 많은 것으로 분석된다. 다만 다른 지역에선 부정적 시선이 적잖았다. 김포시 서울 편입에 대해 알고 있는지를 물어본 결과, 김포 응답자의 98%, 광명 응답자의 91%, 구리 응답자의 92%, 하남 응답자의 98%가 ‘들어본 적 있다’ 등 인식하고 있다는 취지로 답했다. 그러나 김포시 서울 편입을 놓고 다른 지역민들에게 찬반 여부를 묻자, 광명 응답자의 58%는 반대한다고 답해 찬성(28%)의 곱절을 넘었다. 하남에서도 김포시 서울 편입에 대해선 반대 응답이 50%로

Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2252058:
1 이미지 순서 ONhttps://n.news.naver.com/mnews/article/021/0002608753?sid=100 김포시민 “총선서 정부견제” 45% - “정부지원” 36%■ 문화일보·케이스탯 서울편입 여론조사 하남·광명도 정부견제 우위 “與지지보다 편입찬성율 높아” 여권발(發) ‘메가시티’ 구상의 편입 대상으로 거론되는 경기 김포·광명·구리·하남에 거주하는 주민을 대상으로 내년 국회n.news.naver.com2 이미지 순서 ON - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=centristconservatis&no=384018:
1 이미지 순서 ON2 이미지 순서 ON3 이미지 순서 ONhttps://n.news.naver.com/article/021/0002608755?sid=100 김포·구리·하남·광명 절반 이상 “서울편입 총선에 영향줄 것”■ 문화일보·케이스탯 서울편입 여론조사 - 총선 표심 어디로 김포31%·구리31%·하남27% “편입 찬성 후보 지지하겠다” “반대 후보 찍겠다”보다 높아 광명은 찬성-반대 18% 동률 “이슈-후보지지 관계없다”도 4n.news.naver.com


/var/folders/cf/ndy_rrb553z520dr9nryvrww0000gn/T/ipykernel_47410/3541259400.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Contents'] = contents_list


# 101~200

In [4]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Selenium WebDriver를 초기화합니다. ChromeDriver를 사용하는 예시입니다.
driver = webdriver.Chrome()

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 5부터 10까지인 데이터만 필터링
filtered_df = df[df['id'].between(101,200)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    driver.get(link)
    
    # 페이지 로딩을 위해 충분한 시간을 주세요.
    driver.implicitly_wait(10)
    
    # 페이지의 HTML을 가져오기
    page_content = driver.page_source
    
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # div.writing_view_box 부분을 찾아서 가져오기
    writing_view_box = soup.find('div', class_='writing_view_box')
    
    # 가져온 내용을 출력
    if writing_view_box:
        content = writing_view_box.text.strip()
        print(f"Content of {link}:")
        print(content)
        
        # euc-kr로 인코딩 시도하고, 인코딩할 수 없는 문자는 삭제
        try:
            content = content.encode('euc-kr', errors='ignore').decode('euc-kr')
        except UnicodeEncodeError:
            content = None
            print(f"Unable to encode {link} to euc-kr, content deleted.")
        
        # 가져온 내용을 리스트에 추가
        contents_list.append(content)
    else:
        print(f"div.writing_view_box not found in {link}")
        contents_list.append(None)  # 없는 경우에는 None 추가

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장 (euc-kr로 인코딩)
filtered_df.to_csv('/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_(101~200).csv', index=False, encoding='euc-kr')

# Selenium을 종료합니다.
driver.quit()


The chromedriver version (119.0.6045.105) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


Content of https://gall.dcinside.com/mgallery/board/view/?id=joongdo1&no=493459:
https://n.news.naver.com/mnews/article/021/0002608753?sid=100 김포시민 “총선서 정부견제” 45% - “정부지원” 36%■ 문화일보·케이스탯 서울편입 여론조사 하남·광명도 정부견제 우위 “與지지보다 편입찬성율 높아” 여권발(發) ‘메가시티’ 구상의 편입 대상으로 거론되는 경기 김포·광명·구리·하남에 거주하는 주민을 대상으로 내년 국회n.news.naver.com - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2252053:
여권발(發) ‘메가시티’ 구상의 편입 대상으로 거론되는 경기 김포·광명·구리·하남에 거주하는 주민을 대상으로 내년 국회의원 총선거에 관한 인식을 조사한 결과, 김포·광명·하남에서 ‘정부 견제론’이 ‘정부 지원론’보다 우세한 것으로 나타났다. 대통령 국정 운영에 대해서는 네 지역 모두 부정 평가하는 응답이 높게 나타났다. 단 서울시 편입이나 메가시티에 대한 찬성 응답은 총선 인식이나 대통령 국정 지지도보다 높게 나타났다.문화일보가 지난 1∼2일 케이스탯리서치와 함께 김포·광명·구리·하남에 거주 중인 18세 이상 남녀 2009명을 대상으로 여론조사를 한 결과, 내년 총선에서 ‘현 정부를 견제하기 위해 야당 후보가 많이 당선돼야 한다’(정부 견제론)는 응답은 광명 53%, 하남 49%, 김포 45%, 구리 42%로, 모두 40%를 넘었다. 반면 ‘현 정부를 지원하기 위해 여당 후보가 많이 당선돼야 한다’(정부 지원론)는 응답은 구리 42%, 김포 36%, 하남 34%, 광명 32%로, 구리를 제외하고 모두 30%대였다.윤석열 대통령의 국정 운영에 대해서는 네 지역 모두 부정 평가가 긍정 평가의 2배에 달하는 것으로 조사됐다. ‘매우 잘못하고

Content of https://gall.dcinside.com/mgallery/board/view/?id=of&no=960789:
모양부터가 마음안드네딱 봐도 게리멘더링으로선거구 개편 하려는 개수작인데
Content of https://gall.dcinside.com/mgallery/board/view/?id=centristpolitics&no=1517924:
김포를 서울편입 반대할수도없고
Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2248162:
김포가 서울에 편입하기싫어하는거랑 같은이윤가?  - dc official App
Content of https://gall.dcinside.com/board/view/?id=drama_new3&no=12704863:
우리는 국익을 최우선시한다국정운영 개판으로 해서 국민불신감 안겨준 검새 사기꾼은 끌어내린다 전문가 의견 개무시 간신배 들끓어 올바름이란 없다 https://www.youtube.com/watch?v=RxGOscMqoFA?si=D4KIOaEGI94pItY6 [단독] '공매도 금지에 부정적' 김주현 금융위원장 교체 방침 (2023.11.15/뉴스데스크/MBC)다음은 MBC의 단독 보도입니다. 윤석열 대통령이 김주현 금융위원장을 교체할 방침인 것으로 알려졌습니다. 윤 대통령이 어제 직접 주식 공매도 전면 금지의 필요성을 강조한 것과 무관하지 않아 보입니다. 김주현 금융위원장은 당초 공매도 금지에 부정적인 입장을 보인 바 있습니다. htt...youtu.be금융위원장이 전문가인데 공매도금지 반대함 외인 탈출해서 주식시장 불안하다고 했다 의대증원 사실상 낙수효과 없고 건보재정 파탄난다고 건보공단이사장이 말함https://naver.me/GVNMnlJz 건보공단 이사장 “의사 증원하면 건보 지출 많아질 것…‘낙수효과’ 미미”정기석 국민건강보험공단 이사장은 정부의 의대 정원 확대 방침과 관련해 의사 수가 증가하면 불필요한 검

Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4077922:
부산이 ‘2030년 세계박람회(엑스포)’ 유치에 실패하면서 여권이 부산·경남(PK) 민심 동향에 촉각을 곤두세우고 있다. 가뜩이나 “PK 민심이 심상치 않다”는 말이 나오는 가운데 악재가 겹치고 있어서다. 엑스포 유치에 도전한 부산은 지난 28일(현지시간) 오후 프랑스 파리에서 열린 국제박람회기구(BIE) 제173차 총회 1차 투표에서 29표를 얻어 119표를 받은 사우디아라비아 리야드에 완패했다. 총회가 임박할수록 “결선 투표에서 역전이 가능하다”는 희망 섞인 전망이 잇따랐던 터라 예상보다 큰 패배에 여권은 충격에 휩싸였다. 윤석열 대통령도 29일 기자회견에서 “예측이 많이 빗나간 것 같다”며 “이 모든 것은 전부 저의 부족”이라고 고개를 숙였다. PK 지역 국민의힘 의원들은 이날 일제히 우려를 표했다. 부산 지역 중진 의원은 “​그간 부산 유치를 위해 지나치게 공을 들이면서 유치 실패 시 빠져나갈 출구전략이 모호해졌다​”며 “‘​엑스포만 유치하면 다 될 듯이 말해놓고 실패했다’는 원성이 커질까 우려된다​”고 말했다. 초선 의원도 “여권에서 엑스포 부산 유치를 부각해왔는데 큰 표 차로 패배해 지역 민심에 악영향을 줄 것 같아 걱정”이라고 했다. 국민의힘 관계자는 “부산 엑스포 유치 실패는 서울 강서구청장 보궐선거 패배와 같은 무게”라며 “앞으로의 상황에 신중히 대처해야 한다”고 강조했다. 이런 우려 속에 김기현 국민의힘 대표는 30일 당내 부산 지역 의원들을 만나 엑스포 부산 유치 불발에 따른 민심 수습 방안 등을 논의할 예정이다. 사실 엑스포 유치 여부가 결판나기 전 부터 이미 여권은 요동치는 PK 민심을 걱정하던 상황이었다. 지난 24일 발표된 한국갤럽 여론조사에서 윤 대통령의 PK 국정운영 긍정평가는 36%로, 38%인 서울보다 낮았다. 여권 관계자는 “지역별 응답자가 적어 오차범위도 크

Content of https://gall.dcinside.com/mgallery/board/view/?id=joongdo1&no=487386:
https://n.news.naver.com/article/421/0007204673?sid=102 오세훈, 김포·구리·고양 이어 과천시장 만나 '메가시티' 논의권혜정 윤다정 기자 = 오세훈 서울시장은 29일 오전 서울시청 집무실에서 신계용 과천시장을 만나 서울 편입에 대한 과천시의 공식 입장을 들었다. 이번 면담은 김포시장, 구리시장, 고양시장에 이은 네 번째 경기도 지자n.news.naver.com - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=joongdo1&no=487362:
세훈이가 걍 서울이랑 붙어있는 경기도 기초지자체 한번씩만 돌면서 메가서울 편입 계속 얘기하면 그쪽에선 국힘 지지율 오르긴 할듯 어짜피 민주당이 메가시티 법으로 막긴 하겠지만
Content of https://gall.dcinside.com/board/view/?id=immovables&no=6698348:
로마 리야드 다 수도 중심지인데 여기부터 지고들어감 서울에다 유치하고 가용토지 없으니깐 김포편입해서 하면되고 인천국제공항, 김포공항도 가깝고 서울관광도 할수있고 가능성은 더 있었음  - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2239248:
https://n.news.naver.com/mnews/article/421/0007202449 국힘 "메가시티는 제2의 새마을운동…반대 세력은 매국"한상희 노선웅 기자 = 국민의힘은 28일 경기 김포의 서울 편입으로 시작된 메가시티 전략을 '제2의 새마을 운동'에 비유하며 메가시티 구상에 반대하는 세력을 '매국 행위'라고 비판했다. 조경태 뉴시티 프로젝트 특위 n.news.naver.c

Content of https://gall.dcinside.com/board/view/?id=immovables&no=6694860:
서울과 인접한 경기도 일부 도시들을 서울에 편입하는 ‘메가시티’ 이슈가 화제다. 메가시티는 논쟁은 김동연 경기도지사의 공약 중 하나인 경기북부특별자치도 설립 추진에서 시작됐다. 이에 김포시에서 경기북부특별자치도에 편입될 바에야 서울에 편입되겠다는 의견을 내놓으면서 방아쇠가 당겨졌고 10월30일 김기현 국민의힘 대표가 김포시를 서울시로 편입하는 방안을 당론으로 추진하면서 논쟁은 급속도로 확산됐다. 한발 더 나아가 국민의힘 뉴시티 프로젝트 특별위원회는 구리·하남 등 경기도 도시들의 서울 편입 절차를 한 번에 진행하는 ‘행정통합특별법’을 추진해 ‘서울 메가시티’구성에 속도를 더할 방침이다. 서울 중랑구와 바로 인접한 구리의 분위기는 어떨까? 최근 찾은 구리시에서는 서울 편입을 환영하는 현수막을 어렵지 않게 볼 수 있었다. 서울로 출퇴근한다는 구리 시민 A씨는 “구리가 서울로 편입이 되면 면목이나 망우동까지만 가던 시내버스들이 대폭 연장돼 굉장히 편리해질 것 같다”며 “퇴근 후 회식을 할 때면 구리까지 택시를 타고 오는 경우도 있는데 경기도를 넘어가면 추가되는 요금도 사라져 기대가 된다”고 말했다. 구리 시민 B씨는 “구리에도 있을건 다 있어서 서울로 편입돼도 피부로 와닿을 것 같지는 않다”며 “그래도 집값 상승이나 교통망이 확충되면 더 살기 좋아지지 않겠느냐”고 기대감을 내비쳤다. 구리 인창동에서 부동산 공인중개사무소를 운영하는 한 공인중개사는 “구리의 집값이 서울 중랑·노원구 집값에 크게 뒤쳐지지 않는다”며 “서울로 편입이 되면 구리가 중랑·노원구 보다 집값이 더 높아질 수도 있다”고 전했다. 국토교통부 실거래가에 따르면 구리의 대장아파트로 불리는 신명아파트는 국민평형으로 통용되는 전용면적 84㎡가 올 4월 10억 원에 팔렸다. 중랑구 대장아파트로 불리는 사가정 센트럴아이파크의 전용면적 84㎡ 매매 실거래내역을 살펴보면 최저 9억5

Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4070752:
https://n.news.naver.com/article/421/0007200746?sid=100 김기현 "김포 서울 편입 비수도권 차별 아냐…서로 윈윈 게임"한상희 노선웅 기자 = 김기현 국민의힘 대표는 27일 경기 김포시의 서울시 편입이 '비수도권 차별'이라는 야권의 공세에 "전혀 동의할 수 없다"며 "결코 제로섬(한쪽의 이득과 다른 쪽의 손실을 더하면 제로가 되는) n.news.naver.com - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=centristconservatis&no=379591:
https://naver.me/51ajBDKi 오세훈 29일 신계용 과천시장 면담…'메가시티' 논의 이어가오세훈(사진) 서울시장이 신계용 경기도 과천시장과 오는 29일 오전 10시 서울시청에서 과천의 서울 편입 등 ‘메가시티’ 관련 면담을 가질 예정이다. 이번 면담은 지난 6일 김병수 김포시장, 13일 백경현 구리시장,naver.me


/var/folders/cf/ndy_rrb553z520dr9nryvrww0000gn/T/ipykernel_40353/4229636212.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Contents'] = contents_list


# 201~300

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import logging

# 로깅 설정
logger = logging.getLogger(__name__)
logging.basicConfig(filename='crawler.log', level=logging.ERROR)

# Selenium WebDriver를 초기화합니다. ChromeDriver를 사용하는 예시입니다.
driver = webdriver.Chrome()

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 101부터 200까지인 데이터만 필터링
filtered_df = df[df['id'].between(101, 200)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    try:
        driver.get(link)

        # 페이지 로딩을 위해 충분한 시간을 주세요.
        driver.implicitly_wait(10)

        # 페이지의 HTML을 가져오기
        page_content = driver.page_source

        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(page_content, 'html.parser')

        # div.writing_view_box 부분을 찾아서 가져오기
        writing_view_box = soup.find('div', class_='writing_view_box')

        # 가져온 내용을 출력
        if writing_view_box:
            content = writing_view_box.text.strip()
            print(f"Content of {link}:")
            print(content)

            # euc-kr로 인코딩 시도하고, 인코딩할 수 없는 문자는 삭제
            try:
                content = content.encode('euc-kr', errors='ignore').decode('euc-kr')
            except UnicodeEncodeError:
                content = None
                print(f"Unable to encode {link} to euc-kr, content deleted.")

            # 가져온 내용을 리스트에 추가
            contents_list.append(content)
        else:
            print(f"div.writing_view_box not found in {link}")
            contents_list.append(None)  # 없는 경우에는 None 추가

    except (NoSuchElementException, TimeoutException) as e:
        logger.error(f"Error processing link {link}: {str(e)}")

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장 (euc-kr로 인코딩)
output_csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_(101~200).csv'
filtered_df.to_csv(output_csv_path, index=False, encoding='euc-kr')

# Selenium을 종료합니다.
driver.quit()


Content of https://gall.dcinside.com/mgallery/board/view/?id=joongdo1&no=493459:
https://n.news.naver.com/mnews/article/021/0002608753?sid=100 김포시민 “총선서 정부견제” 45% - “정부지원” 36%■ 문화일보·케이스탯 서울편입 여론조사 하남·광명도 정부견제 우위 “與지지보다 편입찬성율 높아” 여권발(發) ‘메가시티’ 구상의 편입 대상으로 거론되는 경기 김포·광명·구리·하남에 거주하는 주민을 대상으로 내년 국회n.news.naver.com - dc official App
Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2252053:
여권발(發) ‘메가시티’ 구상의 편입 대상으로 거론되는 경기 김포·광명·구리·하남에 거주하는 주민을 대상으로 내년 국회의원 총선거에 관한 인식을 조사한 결과, 김포·광명·하남에서 ‘정부 견제론’이 ‘정부 지원론’보다 우세한 것으로 나타났다. 대통령 국정 운영에 대해서는 네 지역 모두 부정 평가하는 응답이 높게 나타났다. 단 서울시 편입이나 메가시티에 대한 찬성 응답은 총선 인식이나 대통령 국정 지지도보다 높게 나타났다.문화일보가 지난 1∼2일 케이스탯리서치와 함께 김포·광명·구리·하남에 거주 중인 18세 이상 남녀 2009명을 대상으로 여론조사를 한 결과, 내년 총선에서 ‘현 정부를 견제하기 위해 야당 후보가 많이 당선돼야 한다’(정부 견제론)는 응답은 광명 53%, 하남 49%, 김포 45%, 구리 42%로, 모두 40%를 넘었다. 반면 ‘현 정부를 지원하기 위해 여당 후보가 많이 당선돼야 한다’(정부 지원론)는 응답은 구리 42%, 김포 36%, 하남 34%, 광명 32%로, 구리를 제외하고 모두 30%대였다.윤석열 대통령의 국정 운영에 대해서는 네 지역 모두 부정 평가가 긍정 평가의 2배에 달하는 것으로 조사됐다. ‘매우 잘못하고

Content of https://gall.dcinside.com/mgallery/board/view/?id=of&no=960789:
모양부터가 마음안드네딱 봐도 게리멘더링으로선거구 개편 하려는 개수작인데
Content of https://gall.dcinside.com/mgallery/board/view/?id=centristpolitics&no=1517924:
김포를 서울편입 반대할수도없고
Content of https://gall.dcinside.com/mgallery/board/view/?id=minjudang&no=2248162:
김포가 서울에 편입하기싫어하는거랑 같은이윤가?  - dc official App
Content of https://gall.dcinside.com/board/view/?id=drama_new3&no=12704863:
우리는 국익을 최우선시한다국정운영 개판으로 해서 국민불신감 안겨준 검새 사기꾼은 끌어내린다 전문가 의견 개무시 간신배 들끓어 올바름이란 없다 https://www.youtube.com/watch?v=RxGOscMqoFA?si=D4KIOaEGI94pItY6 [단독] '공매도 금지에 부정적' 김주현 금융위원장 교체 방침 (2023.11.15/뉴스데스크/MBC)다음은 MBC의 단독 보도입니다. 윤석열 대통령이 김주현 금융위원장을 교체할 방침인 것으로 알려졌습니다. 윤 대통령이 어제 직접 주식 공매도 전면 금지의 필요성을 강조한 것과 무관하지 않아 보입니다. 김주현 금융위원장은 당초 공매도 금지에 부정적인 입장을 보인 바 있습니다. htt...youtu.be금융위원장이 전문가인데 공매도금지 반대함 외인 탈출해서 주식시장 불안하다고 했다 의대증원 사실상 낙수효과 없고 건보재정 파탄난다고 건보공단이사장이 말함https://naver.me/GVNMnlJz 건보공단 이사장 “의사 증원하면 건보 지출 많아질 것…‘낙수효과’ 미미”정기석 국민건강보험공단 이사장은 정부의 의대 정원 확대 방침과 관련해 의사 수가 증가하면 불필요한 검

KeyboardInterrupt: 

# ******************************************

In [11]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

# Selenium WebDriver 초기화
driver = webdriver.Chrome()
driver.set_page_load_timeout(15)  # 페이지 로딩에 대한 최대 대기 시간 설정

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 1부터 100까지인 데이터만 필터링
filtered_df = df[df['id'].between(201, 500)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    try:
        driver.get(link)

        # 페이지 로딩을 위한 대기
        driver.implicitly_wait(10)
    
        # 페이지의 HTML을 가져오기
        page_content = driver.page_source
    
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(page_content, 'html.parser')
    
        # div.writing_view_box 부분을 찾아서 가져오기
        writing_view_box = soup.find('div', class_='writing_view_box')
    
        # 가져온 내용을 출력 및 리스트에 추가
        if writing_view_box:
            content = writing_view_box.text.strip()
            print(f"Content of {link}:")
            print(content)
            contents_list.append(content)
        else:
            print(f"div.writing_view_box not found in {link}")
            contents_list.append(None)

    except TimeoutException:
        print(f"Timeout occurred for {link}. Moving to next link.")
        contents_list.append(None)
        continue

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장
filtered_df.to_csv('/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_(201~500)', index=False, encoding='utf-8')

# Selenium 종료
driver.quit()


Content of https://gall.dcinside.com/board/view/?id=drama_new3&no=12651827:
갑자기 조용해진 듯
Content of https://gall.dcinside.com/mgallery/board/view/?id=canada&no=80534:
요즘 서울 확장 얘기 많던데 서울"시"로 편입하기보다는 메트로벤쿠버나 GTA, 혹은 도쿄도 처럼 메트로 개념을 도입하면 어떨까싶네경기도/수도권을 서울의 메트로라고 보기에는 너무 광범위하고 (벤쿠버의 메트로가 BC가 아니듯이) 수원같이 독자적인 경제권이 있는지역도 많아서 서울에 딱 달라붙은 외곽 시들(김포, 과천, 성남 등등)만 포함해서, 기존서울은 서울특별시 그대로 두고 서울광역권 제정하면 어떨까 싶음메트로벤쿠버나 GTA는 행정적으로 하나의 지역으로 쳐서 통합관리되는것 같던데. 예를들어 엠버 알람도 메트로벤쿠버/비메트로벤쿠버 나뉘고
Content of https://gall.dcinside.com/mgallery/board/view/?id=newconservativeparty&no=4069784:
총선까지 한참 남았는데 병신들만 남아있는게 아닌가 생각이 된다 ㄹㅇ
Content of https://gall.dcinside.com/board/view/?id=immovables&no=6690603:
1. 각종 기피시설 김포 이전2. 김포집값이 서울아파트 평균값을 떨어뜨린다3. 서울 아파트 지을 곳 엄청 확보4 . 서울에 항구 확보5. 한강르네상스 개발 유리오세훈 대통령 유력
Content of https://gall.dcinside.com/mgallery/board/view/?id=conservativefresh&no=955:
(주간조선)조경태 “메가시티는 제2 새마을운동...국가부채 1000조, 뭉쳐야 산다” 1 이미지 순서 ON2 이미지 순서 ON3 이미지 순서 ON4 이미지 순서 ONhttps://www.chosun.com/politics/assem bly/2023/1

Content of https://gall.dcinside.com/mgallery/board/view/?id=monorail&no=437421:
http://news.lghellovision.net/news/articleView.html?idxno=445854 김병수 김포시장 [앵커]김포시의회가 정례회 첫날부터 시끄럽습니다.국민의힘이 단독으로 서울·김포 통합 특별법 찬성 결의안을 채택하는가 하면, 김병수 김포시장은 5호선 연장 노선 직선화 가능성을 언급하기도 했는데요.자세한 소식 김진재 기자가 전해드립니다.[리포트]김포시의회 정례회 첫날, 화두는 지역 최대 현안이 된 서울 편입과 5호선 연장이었습니다.시정질의에 나선 국민의힘 한종우 시의원은 서울 편입과 관련한 질의를 이어가며, 지역 내 혐오시설 설치, 농어촌 특례 폐지 우려에 대해 물었습니다.이에 김병수 김포시장은 편입으로 인해 소각장 등 혐오시설이 설news.lghellovision.net김포 시장은 김포에 5호선을 놓고 싶은 게 아니라 그냥 자기 정치에 이용하는 게 첫번째인 듯..
Content of https://gall.dcinside.com/board/view/?id=immovables&no=6684526:
경기 김포에서 시작된 '서울 메가시티 구상'이 수도권 인접도시로 확산하고 있다. 오세훈 서울시장은 김병수 김포시장, 백경현 구리시장, 이동환 고양시장과 회동한 데 이어 곧 신계용 과천시장을 만난다.24일 서울시와 경기 과천시 등에 따르면 오 시장은 내달 1일(금) 신계용 과천시장을 면담한다. 과천은 오는 24일부터 5일간 만 18세 이상 시민 700명을 대상으로 ‘과천-서울 편입’에 대한 여론조사를 실시하는데, 신 시장은 이날 면담에서 여론조사 결과를 오 시장과 공유하는 것으로 알려졌다.과천은 7개 동으로 구성된 인구 8만의 작은 도시다. 1986년 시흥군 과천면에서 과천시로 승격했다. 읍·면 수준의 작은 지역인 데도 불구하고 '시'의 위상을 유지하고 있는 건 정부제2종합청사를 두고 있는 영향이 

KeyboardInterrupt: 

In [27]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Selenium WebDriver를 초기화합니다.
driver = webdriver.Chrome()

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 5부터 10까지인 데이터만 필터링
filtered_df = df[df['id'].between(201, 300)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    try:
        driver.get(link)
        driver.implicitly_wait(20)  # 대기 시간을 늘림

        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        writing_view_box = soup.find('div', class_='writing_view_box')

        if writing_view_box:
            content = writing_view_box.text.strip()
            contents_list.append(content)
        else:
            print(f"div.writing_view_box not found in {link}")
            contents_list.append(None)
    
    except Exception as e:
        print(f"An error occurred while processing {link}: {e}")
        contents_list.append(None)  # 오류가 발생한 경우 None 추가

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장 (UTF-8로 인코딩)
filtered_df.to_csv('/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_(201,300).csv', index=False, encoding='euc-kr')

# Selenium을 종료합니다.
driver.quit()


/var/folders/cf/ndy_rrb553z520dr9nryvrww0000gn/T/ipykernel_13789/4102427230.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Contents'] = contents_list


UnicodeEncodeError: 'euc_kr' codec can't encode character '\xa0' in position 295: illegal multibyte sequence

In [26]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Selenium WebDriver를 초기화합니다.
driver = webdriver.Chrome()

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 5부터 10까지인 데이터만 필터링
filtered_df = df[df['id'].between(301, 400)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    try:
        driver.get(link)
        driver.implicitly_wait(20)  # 대기 시간을 늘림

        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        writing_view_box = soup.find('div', class_='writing_view_box')

        if writing_view_box:
            content = writing_view_box.text.strip()
            contents_list.append(content)
        else:
            print(f"div.writing_view_box not found in {link}")
            contents_list.append(None)
    
    except Exception as e:
        print(f"An error occurred while processing {link}: {e}")
        contents_list.append(None)  # 오류가 발생한 경우 None 추가

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장 (UTF-8로 인코딩)
filtered_df.to_csv('/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_(301,400).csv', index=False, encoding='euc-kr')

# Selenium을 종료합니다.
driver.quit()

/var/folders/cf/ndy_rrb553z520dr9nryvrww0000gn/T/ipykernel_13789/4002564406.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Contents'] = contents_list


UnicodeEncodeError: 'euc_kr' codec can't encode character '\xa0' in position 302: illegal multibyte sequence

In [21]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Selenium WebDriver를 초기화합니다.
driver = webdriver.Chrome()

# CSV 파일에서 데이터 가져오기
csv_path = '/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_euc_kr.csv'
df = pd.read_csv(csv_path, encoding='euc-kr')

# id가 5부터 10까지인 데이터만 필터링
filtered_df = df[df['id'].between(401, 500)]

# 필터링된 데이터에서 Post Link 가져오기
links = filtered_df['Post Link']

# 새로운 열을 추가할 리스트 초기화
contents_list = []

for link in links:
    try:
        driver.get(link)
        driver.implicitly_wait(20)  # 대기 시간을 늘림

        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        writing_view_box = soup.find('div', class_='writing_view_box')

        if writing_view_box:
            content = writing_view_box.text.strip()
            contents_list.append(content)
        else:
            print(f"div.writing_view_box not found in {link}")
            contents_list.append(None)
    
    except Exception as e:
        print(f"An error occurred while processing {link}: {e}")
        contents_list.append(None)  # 오류가 발생한 경우 None 추가

# 새로운 열 'Contents'를 데이터프레임에 추가
filtered_df['Contents'] = contents_list

# 변경된 데이터프레임을 새로운 CSV 파일로 저장 (UTF-8로 인코딩)
filtered_df.to_csv('/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/dcinside_data_(401,500).csv', index=False, encoding='utf-8')

# Selenium을 종료합니다.
driver.quit()

/var/folders/cf/ndy_rrb553z520dr9nryvrww0000gn/T/ipykernel_13789/3434229654.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Contents'] = contents_list


In [25]:
import pandas as pd

# CSV 파일 경로들
files = [
    "/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_(101~200).csv",
    "/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_(1,100).csv",
    "/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_(201,300).csv",
    "/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_(301,400).csv",
    "/Users/jeonghyeonjeong/Documents/4-1/빅데이터플랫폼/빅플_팀플/dcinside_data_(401,500).csv"
]

# 모든 파일을 읽어 하나의 데이터프레임으로 합치기
combined_df = pd.concat([pd.read_csv(file, encoding='utf-8') for file in files], ignore_index=True)

# 합쳐진 데이터프레임을 새로운 CSV 파일로 저장
combined_df.to_csv("path/to/your/combined_file.csv", index=False)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb1 in position 0: invalid start byte